# Orthogonal Vectors

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Activation, Conv1D, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from art.attacks.evasion import ProjectedGradientDescentL2, DeepFool
from art.metrics import empirical_robustness
from art.defences.trainer import AdversarialTrainer

In [ ]:
def create_neural_net(input_size, hidden_layers, hidden_size, num_classes, no_final=False):
    model = Sequential()
    model.add(BatchNormalization(input_shape=(input_size,)))
    for _ in range(hidden_layers):
        model.add(Dense(hidden_size))
        model.add(Activation('relu'))
    model.add(Dense(num_classes))
    if not no_final:
        model.add(Activation('tanh'))
    return model


In [ ]:



def create_cnn_one_d(hidden_channels=1024, kernel_size=5, num_classes=1, padding='same', no_final=False):
    model = Sequential()
    model.add(Conv1D(hidden_channels, kernel_size, padding=padding))
    model.add(Activation('relu'))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(num_classes))
    if not no_final:
        model.add(Activation('tanh'))
    return model

def train_model(model, data, labels, epochs=1000):
    model.compile(optimizer=Adam(lr=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(data, labels, batch_size=data.shape[0], epochs=epochs)
    return model

def test_pgd(model, data, labels, epsilon, res_row):
    attack = ProjectedGradientDescentL2(estimator=model, eps=epsilon, eps_step=epsilon/50, max_iter=100)
    perturbed_data = attack.generate(x=data, y=labels)
    new_out = model(perturbed_data)
    pred = tf.argmax(new_out, axis=1)
    correct = tf.reduce_sum(tf.cast(tf.equal(pred, labels), tf.float32))
    res_row.append(100 * correct / labels.shape[0])

def test_deepfool(model, data, labels, res_row):
    attack = DeepFool(estimator=model)
    perturbed_data = attack.generate(x=data, y=labels)
    new_out = model(perturbed_data)
    pred = tf.argmax(new_out, axis=1)
    correct = tf.reduce_sum(tf.cast(tf.equal(pred, labels), tf.float32))
    res_row.append(100 * correct / labels.shape[0])

def add_linear_separator(d1, d2, p):
    z_1 = np.mean(d1, axis=0)
    z_2 = np.mean(d2, axis=0)
    z_1 /= np.linalg.norm(z_1)
    z_2 /= np.linalg.norm(z_2)

    d1_n = z_1 * p + d1 * (1 - p)
    d2_n = z_2 * p + d2 * (1 - p)
    d1_n /= np.expand_dims(np.linalg.norm(d1_n, axis=1), 1)
    d2_n /= np.expand_dims(np.linalg.norm(d2_n, axis=1), 1)
    return d1_n, d2_n

def sample_orth_datasets(dimension, samples, p):
    if p < 0:
        d1, d2 = x[:k], x[dimension//2:dimension//2+samples]
    else:
        d1, d2 = x[:k], x[dimension//2:dimension//2+samples]
        z1, z2 = x[k-1], x[dimension//2+samples-1]
        d1 = d1 + z1 * p
        d2 = d2 + z2 * p
    return d1, d2
